In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('stocks.csv')[['ticker','date','ret']]

In [3]:
df.date = df.date//100

In [4]:
df = df[df.date>201000]

In [5]:
df['ret'] = df['ret']*100

In [6]:
n = 28

In [7]:
sample = []
for stock in set(df.ticker):
        x = df[df.ticker==stock].set_index('date').dropna()
        if len(x)>120:
            sample.append(x[['ret']].rename({'ret':stock},axis=1))

In [8]:
df = pd.concat(sample,axis=1)

In [9]:
w = [1/n]*n

$ w^t \times V \times w \$

In [10]:
# portfolio variance
std = np.sqrt(df.cov().mul(w,axis=0).mul(w,axis=1).sum().sum())

eret = df.mul(w,axis=1).sum(axis=1).mean()*12

In [11]:
np.random.normal(20)

18.796539350979756

In [12]:
w = np.random.rand(n)

w = w / w.sum()

In [16]:
def ret_var(w):
    # below code is to use variance-covariance matrix
    # std = np.sqrt(df.cov().mul(w,axis=0).mul(w,axis=1).sum().sum())*np.sqrt(12)
    
    # below code is to use std of monthly portfolio return
    std2 = df.mul(w,axis=1).sum(axis=1).std()*np.sqrt(12)
    eret = df.mul(w,axis=1).sum(axis=1).mean()*12
    return eret,std,w

In [ ]:
list_of_retvar = []
for _ in range(5000):
    w = np.random.rand(n) - np.random.rand(n)
    w = w / w.sum()
    if w.max()<3 and w.min()>=-1:
        list_of_retvar.append(ret_var(w))    

In [ ]:
ports = pd.DataFrame(list_of_retvar,columns=['ret','std','w'])
ports.plot(x='std',y='ret',kind='scatter')#.plot(x='std',y='ret',kind='scatter',xticks=range(10),yticks=range(25))

In [ ]:
ports['sharpe'] = (ports['ret'] - 1.6)/ports['std']

ports.sort_values('sharpe')

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
def compute(_):
    r = np.random.RandomState()
    w = r.rand(n) - r.rand(n)
    w = w / w.sum()
    if w.max()<3 and w.min()>=-1:
        return ret_var(w)

In [ ]:
with ProcessPoolExecutor() as p:
    ports = pd.DataFrame(p.map(compute,range(1000000)),columns=['ret','std','w'])

In [ ]:
ports['sharpe'] = (ports['ret'] - 1.6)/ports['std']
ports.dropna().sort_values('sharpe')